In [1]:
import pandas as pd
from fedot.api.main import Fedot
from fedot.core.pipelines.pipeline_builder import PipelineBuilder

In [2]:
path = "../datasets/tabular-playground-series-may-2022"

train = pd.read_csv(f'{path}/train.csv')
test = pd.read_csv(f'{path}/test.csv')
sub = pd.read_csv(f'{path}/sample_submission.csv')

In [3]:
train.drop(columns=["id", "f_27"], inplace=True)
test.drop(columns=["id", "f_27"], inplace=True)

In [4]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(train, train_size=0.85, random_state=42)

In [7]:
auto_model = Fedot(
    problem="classification",
    metric=["precision", "accuracy", "roc_auc"],
    preset="best_quality",
    with_tuning=True,
    timeout=2,
    cv_folds=10,
    seed=42,
    n_jobs=4,
    logging_level=10,
    initial_assumption=PipelineBuilder()
    .add_node(
        "lgbm",
    )
    .build(),
    use_pipelines_cache=False,
    use_auto_preprocessing=True
)

2024-06-24 16:30:51,242 - Unknown integration target: 
2024-06-24 16:30:55,436 - Unknown integration target: 
2024-06-24 16:30:59,790 - Unknown integration target: 
2024-06-24 16:31:03,912 - Unknown integration target: 
2024-06-24 16:31:08,096 - Unknown integration target: 
2024-06-24 16:31:17,095 - Unknown integration target: 
2024-06-24 16:31:26,044 - Unknown integration target: 
2024-06-24 16:31:35,054 - Unknown integration target: 
2024-06-24 16:31:36,131 - Unknown integration target: 
2024-06-24 16:31:36,133 - Unknown integration target: 


In [8]:
auto_model.fit(features=train, target="target")

2024-06-24 16:38:45,013 - ApiDataProcessor - Preprocessing data
2024-06-24 16:38:45,018 - ApiDataProcessor - Train Data (Original) Memory Usage: 175.09 MB Data Shapes: ((765000, 30), (765000, 1))
2024-06-24 16:40:35,841 - ApiDataProcessor - Train Data (Processed) Memory Usage: 186.77 MB Data Shape: ((765000, 32), (765000, 1))
2024-06-24 16:40:35,845 - ApiDataProcessor - Data preprocessing runtime = 0:01:50.831650
2024-06-24 16:41:02,776 - AssumptionsHandler - Initial pipeline fitting started
2024-06-24 16:41:04,710 - PipelineNode - Trying to fit pipeline node with operation: lgbm
2024-06-24 16:41:46,139 - PipelineNode - Obtain prediction in pipeline node by operation: lgbm
2024-06-24 16:41:47,702 - AssumptionsHandler - Initial pipeline was fitted successfully
2024-06-24 16:41:47,706 - AssumptionsHandler - Memory consumption for fitting of the initial pipeline in main session: current 486.9 MiB, max: 1941.0 MiB
2024-06-24 16:41:47,710 - ApiComposer - Initial pipeline was fitted in 71.9 

{'depth': 1, 'length': 1, 'nodes': [lgbm]}

In [9]:
prediction = auto_model.predict_proba(features=test)

2024-06-24 16:54:33,010 - PipelineNode - Obtain prediction in pipeline node by operation: lgbm


In [10]:
sub.target = prediction.ravel()
sub.to_csv("submission.csv", index=False)

In [11]:
prediction_valid = auto_model.predict_proba(features=valid)

2024-06-24 16:54:58,880 - PipelineNode - Obtain prediction in pipeline node by operation: lgbm


In [13]:
from sklearn.metrics import roc_auc_score

roc_auc_score(valid.target, prediction_valid)

0.8595260114096817